In [259]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [260]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [261]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# comment this out when stuff actually works
bases = ['X', 'Z'] # these are all commuting
coefficients = [1]*len(bases)
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [262]:
hamiltonian

array([[ 1.+0.j,  1.+0.j],
       [ 1.+0.j, -1.+0.j]])

In [263]:
is_hermitian(hamiltonian)

0.0

In [264]:
def get_circuit(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            b = b[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
    return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [265]:
get_circuit(n=1).draw()

░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░  ░  ░ ┌───────┐ ░  ░ 
q: ─░─┤ H ├─░─┤ Rz(2) ├─░─┤ H ├─░──░──░─┤ Rz(2) ├─░──░─
    ░ └───┘ ░ └───────┘ ░ └───┘ ░  ░  ░ └───────┘ ░  ░

In [290]:
e_ith, U_circuit = get_unitaries(n=100)

In [291]:
def get_error(e_ith, U_circuit):
    diff = U_circuit.data - e_ith
    # measure the operator's accuracy
    w, v = LA.eig(diff)

    return np.absolute(max(w))

get_error(e_ith, U_circuit)

0.006984665289642003

In [268]:
np.around(e_ith, decimals=3)

array([[0.156-0.698j, 0.   -0.698j],
       [0.   -0.698j, 0.156+0.698j]])

In [269]:
np.around(U_circuit, decimals=3)

array([[ 0.156-0.698j, -0.007-0.698j],
       [ 0.007-0.698j,  0.156+0.698j]])

In [270]:
hamiltonian

array([[ 1.+0.j,  1.+0.j],
       [ 1.+0.j, -1.+0.j]])

In [271]:
e_ith-U_circuit.data

array([[-1.16411262e-05-1.87614788e-05j,  6.98460519e-03-1.87614788e-05j],
       [-6.98460519e-03-1.87614788e-05j, -1.16411262e-05+1.87614788e-05j]])

# New Stuff

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

In [287]:
def buildPauliString(qc, pauliString, t, n):
    c, b = pauliString
    # implement e^{-i*c*b*t/n}
    b = b[::-1]
    q = []
    qc.barrier()
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.sdg(i)
            qc.h(i)
            q.append(i)
        elif (op == 'I'):
            continue
    qc.barrier()
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t/n, q[-1])
        if (len(q) > 1):
            for i in reversed(range(len(q)-1)):
                qc.cx(q[i], q[-1])
        qc.barrier()
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.h(i)
                qc.s(i)
            elif (op == 'I'):
                continue
    qc.barrier()
    return qc

def buildHamiltonian(qc, pauliStrings, t, n):
    for pauliString in pauliStrings:
        # print(pauliString[1])
        qc = buildPauliString(qc, pauliString, t, n)
    return qc

In [288]:
# Higher order trotter formulation.
def testMethods(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n)
        
    e_ith = expm(-1.j * t * hamiltonian)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return get_error(e_ith, U_circuit)

def get_circuit_2(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        for pauliString in pauliStrings:
            print(f"{pauliString[1]} -> ")
            qc = buildPauliString(qc, pauliString, t, n * 2)
            qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2)
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# Higher order trotter formulation.
def get_circuit_3(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n * 2)
        qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2 / len(pauliStrings))
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

def get_unitaries_2(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_2(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

def get_unitaries_3(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_3(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [289]:
testMethods(n=100)

0.006984665289642003

In [275]:
get_circuit_2(n=2).draw()

X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X


░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░  ░ ┌─────────┐ ░  ░  ░ ┌───┐ ░ »
q: ─░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░──░─┤ Rz(0.5) ├─░──░──░─┤ H ├─░─»
    ░ └───┘ ░ └─────────┘ ░ └───┘ ░  ░  ░ └─────────┘ ░  ░  ░ └───┘ ░ »
«   ┌─────────┐ ░ ┌───┐ ░  ░  ░ ┌─────────┐ ░  ░  ░  ░ ┌─────────┐ ░  ░  ░ »
«q: ┤ Rz(0.5) ├─░─┤ H ├─░──░──░─┤ Rz(0.5) ├─░──░──░──░─┤ Rz(0.5) ├─░──░──░─»
«   └─────────┘ ░ └───┘ ░  ░  ░ └─────────┘ ░  ░  ░  ░ └─────────┘ ░  ░  ░ »
«   ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░  ░ »
«q: ┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░──░─»
«   └───┘ ░ └─────────┘ ░ └───┘ ░  ░ └───┘ ░ └─────────┘ ░ └───┘ ░  ░  ░ »
«   ┌─────────┐ ░  ░  ░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░  ░ ┌─────────┐ ░  ░ »
«q: ┤ Rz(0.5) ├─░──░──░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░──░─┤ Rz(0.5) ├─░──░─»
«   └─────────┘ ░  ░  ░ └───┘ ░ └─────────┘ ░ └───┘ ░  ░  ░ └─────────┘ ░  ░ »
«    ░  ░ ┌─────────┐ ░  ░  ░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░ 
«q: ─░──░─┤ Rz(0.5) ├─░──░──░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░─
«    ░  ░ └─────────┘ ░  ░  ░ └───┘ ░ └─────────┘ ░ └───┘ ░

In [276]:
e_ith1, U_circuit1 = get_unitaries_2(n=100)

X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X
X -> 
Z
X
Z -> 
Z
X


In [277]:
get_error(e_ith1, U_circuit1)

0.6924528299000203

In [278]:
get_circuit_3(n=2).draw()

X
Z
Z
X
X
Z
Z
X


░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░  ░ ┌─────────┐ ░  ░  ░  ░ ┌───────┐ ░ »
q: ─░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░──░─┤ Rz(0.5) ├─░──░──░──░─┤ Rz(1) ├─░─»
    ░ └───┘ ░ └─────────┘ ░ └───┘ ░  ░  ░ └─────────┘ ░  ░  ░  ░ └───────┘ ░ »
«    ░  ░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░  ░ ┌───┐ ░ ┌─────────┐ ░ ┌───┐ ░  ░  ░ »
«q: ─░──░─┤ H ├─░─┤ Rz(1) ├─░─┤ H ├─░──░─┤ H ├─░─┤ Rz(0.5) ├─░─┤ H ├─░──░──░─»
«    ░  ░ └───┘ ░ └───────┘ ░ └───┘ ░  ░ └───┘ ░ └─────────┘ ░ └───┘ ░  ░  ░ »
«   ┌─────────┐ ░  ░  ░  ░ ┌───────┐ ░  ░  ░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░ 
«q: ┤ Rz(0.5) ├─░──░──░──░─┤ Rz(1) ├─░──░──░─┤ H ├─░─┤ Rz(1) ├─░─┤ H ├─░─
«   └─────────┘ ░  ░  ░  ░ └───────┘ ░  ░  ░ └───┘ ░ └───────┘ ░ └───┘ ░

In [279]:
e_ith2, U_circuit2 = get_unitaries_3(n=100)

X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X
X
Z
Z
X


In [280]:
get_error(e_ith2, U_circuit2)

0.6924374722714909

In [281]:
np.around(e_ith2, decimals=3)

array([[0.156-0.698j, 0.   -0.698j],
       [0.   -0.698j, 0.156+0.698j]])

In [282]:
np.around(U_circuit2, decimals=3)

array([[-0.523-0.603j,  0.003-0.603j],
       [-0.003-0.603j, -0.523+0.603j]])